In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import os
import shutil
import pandas as pd

cudnn.benchmark = True
plt.ion()

# **Restructuring the dataset in a new folder with the csv above and the images**

In [10]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
pysio_paths = []
for dirname, _, filenames in os.walk('/kaggle/input/physionet-flash-data/data/records500'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if os.path.isfile(os.path.join(dirname, filename)):
            pysio_paths.append(os.path.join(dirname, filename))
        

In [11]:
print(len(pysio_paths))

65397


In [14]:


def extract_info_from_hea(label_mapping):
    data = []

    # Iterate over all files in the given folder
    for filename in pysio_paths:
        if filename.endswith(".hea"):
            file_path = filename #os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                lines = file.readlines()
                image_path = ""
                labels = ""
                
                # Iterate over each line in the .hea file
                for line in lines:
                    if 'Labels' in line:
                        labels = line.split(":")[1].strip()
                    if 'png' in line:
                        image_path = line.split(":")[1].strip()
                
                # Create a dictionary to hold the image path and label information
                label_info = {label: 0 for label in label_mapping}
        
                if labels:
                    for label in labels.split(","):
                        if label.strip() in label_mapping:
                            label_info[label.strip()] = 1
                
                label_info["Image_Name"] = image_path
                label_info["Image_Path"] = filename
                data.append(label_info)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

In [15]:
# Define the label mapping
label_mapping = {"NORM", "Acute MI", "Old MI", "STTC", "CD", "HYP", "PAC", "PVC", "AFIB/AFL", "TACHY", "BRADY"}

# Extract information from the .hea files and create the DataFrame
df = extract_info_from_hea(label_mapping)

In [17]:
df.head()

,Acute MI,PAC,AFIB/AFL,CD,BRADY,Old MI,TACHY,PVC,NORM,STTC,HYP,Image_Name,Image_Path
0,0,0,0,0,0,1,0,0,0,1,0,21240_hr-0.png,/kaggle/input/physionet-flash-data/data/record...
1,0,0,0,0,0,1,0,0,0,1,0,21604_hr-0.png,/kaggle/input/physionet-flash-data/data/record...
2,0,0,0,0,0,0,0,0,1,0,0,21080_hr-0.png,/kaggle/input/physionet-flash-data/data/record...
3,0,0,0,0,0,0,0,0,1,0,0,21694_hr-0.png,/kaggle/input/physionet-flash-data/data/record...
4,0,0,0,0,0,0,0,1,0,1,1,21504_hr-0.png,/kaggle/input/physionet-flash-data/data/record...


In [18]:
csv_path ='multilabel_classification.csv'
df.to_csv(csv_path, index=False)

# **Prepare the "physionet-100-00000" by adding the different labels to the header file**


In [2]:
# !git clone https://github.com/physionetchallenges/python-example-2024.git

fatal: destination path 'python-example-2024' already exists and is not an empty directory.


In [16]:
# !python "/kaggle/working/python-example-2024/prepare_image_data.py" -i "/kaggle/input/physionet-100-00000/content/data/ptb-103/records100/00000" -o "/kaggle/working/records100/00000"

# then run the script above to save the files in the csv using the script above.